# Factor de Friccion con PySR

In [1]:
## Importando Librerias 
import numpy as np
import pandas as pd
from pysr import PySRRegressor
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


### Cargando datos

In [2]:
# cargando datos

# importacion de DF
file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_ff_25.txt'
df_ff_25 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_ff_53.txt'
df_ff_53 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_ff_74.txt'
df_ff_74 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_ff_102.txt'
df_ff_102 = pd.read_csv(file_path, delimiter=',')

In [4]:
# definiendo conjuntos de train y test

# definiendo conjunto de train
df_ff_train = pd.concat([df_ff_25, df_ff_74], ignore_index=True)

# separando entre x e y
y_train = df_ff_train.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_train = df_ff_train.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','colIndex','frictionFactor'])

# definiendo conjunto de test

y_test = df_ff_53.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_test = df_ff_53.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','colIndex','frictionFactor'])

# renombrando
X_train.rename(columns={'Rem':'Re'}, inplace=True)
X_test.rename(columns={'Rem':'Re'}, inplace=True)

### Entrenando Modelo

In [13]:
model = PySRRegressor(
    niterations=100,
    binary_operators=["+", "*", "pow"],
    unary_operators=[],
    population_size=200,
    verbosity=1,
    maxsize=12,
    parsimony=1e-3,
    progress=True
)

model.fit(X_train, y_train)

d:\env\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
d:\env\lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(
d:\env\lib\site-packages\pysr\sr.py:73: UserWarning: You are using the `^` operator, but have not set up `constraints` for it. This may lead to overly complex expressions. One typical constraint is to use `constraints={..., '^': (-1, 1)}`, which will allow arbitrary-complexity base (-1) but only powers such as a constant or varia


Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 1500 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 4.180e+01
Head worker occupation: 0.0%
Progress: 1 / 1500 total iterations (0.067%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
3           5.588e+00  5.314e+00  y = -2.4264 * -1.644
5           2.871e+00  3.331e-01  y = 2.5313 + (K ^ -1.4303)
7           9.306e-01  5.632e-01  y = (2.5313 * (K ^ -1.4303)) + 0.9818
9           8.920e-01  2.119e-02  y = (K ^ -1.1974) + (2.4334 * (K ^ -1.2183))
-------------------------------

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                           4.239675   
	1  >>>>  0.455932                        3.4347172 * (K ^ -1.209575)   
	2        0.001579          ((0.06762905 + K) ^ -1.339577) * 3.739836   
	3        0.136316  (7.810259 * (K ^ -1.2095693)) * (Re ^ -0.10515...   
	4        0.030612  ((5.919568 * ((Re ^ K) ^ -0.3994728)) * 4.9490...   
	
	       loss  complexity  
	0  5.525302           1  
	1  0.891910           5  
	2  0.889098           7  
	3  0.676935           9  
	4  0.636734          11  
]

In [15]:
best_equation = model.equations_.iloc[3]['equation']
print(best_equation)

(7.810259 * (K ^ -1.2095693)) * (Re ^ -0.10515685)


### Pruebas

In [9]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 0.8335812597548942
R^2: 0.8531704688664192
